## reward model的训练过程
2025-10-04 23:50:24 Saturday

参考 https://blog.csdn.net/shizheng_Li/article/details/145947974



## 1. 应用场景

训练一个 **Reward Model ($R_\phi(x, y)$)**，用于评估模型输出 (y) 对输入 (x) 的“人类偏好得分”。
在强化学习阶段（如 **PPO/GRPO**）中提供奖励信号。


## 2. 实现方式

* **模型结构**
  复用经过 SFT 的语言模型骨干（已具备较好语言理解能力），
  将原本的 *下一词预测头*（LM Head）替换为 **回归头**（输出单个标量 reward 值）。

* **训练数据结构**
  每个样本包含一对人类偏好：
  $(x, y^+, y^-)$
  其中：

  * (x)：prompt（问题/输入）
  * ($y^+$)：更优回答（human preferred）
  * ($y^-$)：较差回答（human rejected）

* **损失函数（Bradley–Terry 形式）**
  $$L(\phi) = -\mathbb{E}\big[\log \sigma(R_\phi(x, y^+) - R_\phi(x, y^-))\big]$$
  目标是最大化 $(R_\phi(x, y^+) > R_\phi(x, y^-))$ 的概率。



## 3. 训练流程总结

1. **加载预训练的 SFT 模型**
   （具备较好语言理解与生成能力）
2. **替换输出层** → 线性回归头（1个神经元输出 reward）
3. **构造偏好数据对** ((x, $y^+, y^-$))
4. **拼接输入**：“`x + y`” 作为模型输入，reward 对应整个回答
5. **前向计算** 得到 $(R(x, y^+))、(R(x, y^-))$
6. **计算对比损失**（希望 better > worse）
7. **反向传播 + 优化参数**


## 4.reward model的训练代码实现

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
from icecream import ic
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载 tokenizer 并确保 pad_token 存在（GPT2 等可能没有）
tokenizer = AutoTokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

class RewardModel(nn.Module):
    def __init__(self, tokenizer, model_name='gpt2', device=None):
        super().__init__()
        self.tokenizer = tokenizer
        self.device = device or torch.device('cpu')
        self.backbone = AutoModel.from_pretrained(model_name).to(self.device)       ##假设为已经有的sft模型
        self.reward_head = nn.Linear(self.backbone.config.hidden_size, 1).to(self.device)

    def forward(self, prompt: str, response: str):
        prompt_ids = self.tokenizer.encode(prompt, add_special_tokens=False)
        response_ids = self.tokenizer.encode(response, add_special_tokens=False)
        sep = self.tokenizer.eos_token_id if self.tokenizer.eos_token_id is not None else None
        if sep is not None:
            input_ids = prompt_ids + [sep] + response_ids
            prompt_len = len(prompt_ids) + 1
        else:
            input_ids = prompt_ids + response_ids
            prompt_len = len(prompt_ids)

        # 截断到模型支持的最大长度（从右侧保留 response 的结尾）
        max_len = getattr(self.backbone.config, 'n_positions', None) or getattr(self.backbone.config, 'max_position_embeddings', None)
        if max_len is not None and len(input_ids) > max_len:
            excess = len(input_ids) - max_len
            input_ids = input_ids[excess:]
            prompt_len = max(0, prompt_len - excess)

        input_ids_tensor = torch.tensor([input_ids], dtype=torch.long, device=self.device)
        attention_mask = torch.ones_like(input_ids_tensor, device=self.device)
        ic(input_ids_tensor.shape,attention_mask.shape)
        outputs = self.backbone(input_ids=input_ids_tensor, attention_mask=attention_mask, return_dict=True)
        last_hidden = outputs.last_hidden_state  # (B, L, D)

        seq_len = last_hidden.size(1)
        if prompt_len >= seq_len or prompt_len < 0:
            pooled = last_hidden[:, -1, :]
        else:
            response_hidden = last_hidden[:, prompt_len:, :]
            pooled = response_hidden.mean(dim=1)

        reward = self.reward_head(pooled).squeeze(-1)
        return reward

# 简单示例数据（文本）
samples = [
    { 'prompt': 'Explain the concept of overfitting in machine learning.', 
     'chosen': 'Overfitting happens when a model learns noise instead of the true pattern.', 
     'rejected': 'It means the model trains too much.' },
    { 'prompt': 'What is the capital of France?', 
     'chosen': 'The capital of France is Paris.',
       'rejected': 'France is in Europe.' },
]

# 初始化模型与优化器
model = RewardModel(tokenizer=tokenizer, model_name='gpt2', device=device)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# 训练循环（仅作演示，样本很少）
for epoch in range(3):
    total_loss = 0.0
    for sample in samples:
        r_chosen = model(sample['prompt'], sample['chosen'])
        r_rejected = model(sample['prompt'], sample['rejected'])
        loss = -torch.log(torch.sigmoid(r_chosen - r_rejected)).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch}, Loss = {total_loss:.4f}')


ic| input_ids_tensor.shape: torch.Size([1, 26])
    attention_mask.shape: torch.Size([1, 26])
ic| input_ids_tensor.shape: torch.Size([1, 20])
    attention_mask.shape: torch.Size([1, 20])
ic| input_ids_tensor.shape: torch.Size([1, 15])
    attention_mask.shape: torch.Size([1, 15])
ic| input_ids_tensor.shape: torch.Size([1, 13])
    attention_mask.shape: torch.Size([1, 13])
ic| input_ids_tensor.shape: torch.Size([1, 15])
    attention_mask.shape: torch.Size([1, 15])
ic| input_ids_tensor.shape: torch.Size([1, 13])
    attention_mask.shape: torch.Size([1, 13])


Epoch 0, Loss = 2.4433


ic| input_ids_tensor.shape: torch.Size([1, 26])
    attention_mask.shape: torch.Size([1, 26])
ic| input_ids_tensor.shape: torch.Size([1, 20])
    attention_mask.shape: torch.Size([1, 20])
ic| input_ids_tensor.shape: torch.Size([1, 15])
    attention_mask.shape: torch.Size([1, 15])
ic| input_ids_tensor.shape: torch.Size([1, 13])
    attention_mask.shape: torch.Size([1, 13])
ic| input_ids_tensor.shape: torch.Size([1, 15])
    attention_mask.shape: torch.Size([1, 15])
ic| input_ids_tensor.shape: torch.Size([1, 13])
    attention_mask.shape: torch.Size([1, 13])


Epoch 1, Loss = 1.0154


ic| input_ids_tensor.shape: torch.Size([1, 26])
    attention_mask.shape: torch.Size([1, 26])
ic| input_ids_tensor.shape: torch.Size([1, 20])
    attention_mask.shape: torch.Size([1, 20])
ic| input_ids_tensor.shape: torch.Size([1, 15])
    attention_mask.shape: torch.Size([1, 15])
ic| input_ids_tensor.shape: torch.Size([1, 13])
    attention_mask.shape: torch.Size([1, 13])
ic| input_ids_tensor.shape: torch.Size([1, 15])
    attention_mask.shape: torch.Size([1, 15])
ic| input_ids_tensor.shape: torch.Size([1, 13])
    attention_mask.shape: torch.Size([1, 13])


Epoch 2, Loss = 0.6366


## 5.reward model的损失函数和 DPO很像，那两者有什么区别?

两者只是形式上像，但是谁在被训练是不一样的。



- Reward Model 用人类偏好数据 ((x, y^+, y^-)) 训练：
    $$L_{\text{RM}}(\phi)
    = - \mathbb{E}_{(x, y^+, y^-)} \left[
    \log \sigma\big(R_\phi(x, y^+) - R_\phi(x, y^-)\big)
    \right]$$

    其中：

    * ($R_\phi(x, y)$)：Reward Model 输出的分数；
    * ($\sigma(z) = 1 / (1 + e^{-z})$)：sigmoid；
    * 优化参数是 **Reward Model 参数** $(\phi)$。

    ➡️ 损失的含义是让「更好」回答的得分高于「更差」回答。故而梯度更新的 ( $R_{\phi}$ )；




- DPO 损失函数形式：
    $$L_{\text{DPO}}(\theta) = - \mathbb{E}_{(x, y^+, y^-)} \left[
    \log \sigma\left(
    \beta \cdot \left(
    \log\frac{\pi_\theta(y^+|x)}{\pi_\text{ref}(y^+|x)} - \log \frac{\pi_{\theta}(y^-|x)}{\pi_{\text{ref}}(y^-|x)} 
    \right)
    \right)
    \right]$$


    其中：

    * $(\pi_\theta$)：正在训练的语言模型；
    * $(\pi_{\text{ref}})$：参考模型（通常是SFT模型）；
    * $(\beta)$：温度系数，控制对偏好的敏感程度。


    ➡️ DPO是不需要显式地教一个模型打分了，而是直接优化生成模型，让它的输出顺序符合这些人类偏好。故而梯度更新的是$\pi_{\theta}$。



